In [1]:
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import (train_test_split, GridSearchCV, cross_val_score)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, OneHotEncoder, FunctionTransformer, KBinsDiscretizer)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import (ElasticNet, Ridge, Lasso)
from scipy import stats
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# constants and helper methods

CONDITIONS_DICT = {"NA": 0, "NaN": 0, "nan": 0, "Po": 2, "Fa": 3, "TA": 4, "Gd":6, "Ex": 10}

# constants
CATEGORY_LABELS = {"KitchenQual":       CONDITIONS_DICT,
                    "GarageCond":       CONDITIONS_DICT,
                    "GarageQual":       CONDITIONS_DICT,
                    "ExterQual":        CONDITIONS_DICT,
                    "ExterCond":        CONDITIONS_DICT,
                    "BsmtQual":         CONDITIONS_DICT,
                    "BsmtCond":         CONDITIONS_DICT,
                    "FireplaceQu" :     CONDITIONS_DICT,
                    "HeatingQC" :       CONDITIONS_DICT,
                    "LotConfig":     {"Inside": 0, "Corner": 6, "CulDSac": 10, "FR2": 3, "FR3":4},
                    "Utilities":     {"ELO": 0, "NoSeWa": 1, "NoSewr": 2, "AllPub": 3},
                    "LandSlope":     {"Gtl": 10, "Mod": 4, "Sev": 1},
                    "LotShape":     {"Reg": 10, "IR1": 5, "IR2": 3, "IR3": 1},
                    "GarageType":     {"NA": 0, "nan": 0, "Basment": 4,  "Detchd": 1, "CarPort": 3, "BuiltIn": 5, "Attchd": 7, "2Types": 12},
                    "BldgType":     {"TwnhsI": 1, "Twnhs": 2, "TwnhsE": 3, "Duplex": 5,  "2fmCon": 7, "1Fam": 12},
                    "CentralAir":     {"N": 1, "Y": 10},
                    "Electrical":     {"Mix": 1, "FuseP": 3, "FuseF": 5,  "FuseA": 7, "SBrkr": 12},
                    "MSZoning":     {"RL": 100, "RM": 60, "C (all)": 20, "FV": 30, "RH": 30},
                    "LandContour":     {"Lvl": 100, "Low": 15, "Bnk": 25, "HLS": 5},
                    "Fence":     {"NA": 0, "MnPrv": 25, "MnWw": 15, "GdWo": 40, 'GdPrv': 100},
                    "Functional":     {"Typ": 100, "Min1": 70, "Min2": 50, "Mod": 40, "Maj1": 25, "Maj2": 20, "Min2": 10, "Sev": 5, "Sal": 1},
                    "MiscFeature":     {"NA": 0, "Shed": 30, "Gar2": 40, "Othr": 25, "TenC": 100},
                    "PavedDrive":     {"Y": 100, "P": 30, "N": 0},
                    }

CAT_COLS_TO_IGNORE = ["Functional",
                        "MiscFeature",
                        "Electrical",
                        "Fence",
                        "FireplaceQu",
                        "HeatingQC"           
                    ]

CAT_COLS = [ x for x in CATEGORY_LABELS.keys() if x not in CAT_COLS_TO_IGNORE]
print(CAT_COLS)

# plot correlations
def plotCoorelations(df):
    # remove non_numeric features  
    corr = df.corr()
    corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# define a method to use Isolation Forest for outlier detection
def outlierDetect_IsolationForest(X, outlierFraction = 0.02):
    clf = IsolationForest( behaviour = 'new', contamination = outlierFraction)
    preds = clf.fit_predict(X)
    return np.where(preds == -1)

# define a method to use Isolation Forest for outlier detection
def outlierDetect_ZScore(X, zValue = 3):
    z = np.abs(stats.zscore(X))
    return np.where(z > zValue)
    
def displayScoresExp1p(scores):
    print("Scores:", np.expm1(scores))
    print("Mean:", np.expm1(scores.mean()))
    print("standard deviation:", np.expm1(scores.std()))
    
def displayScores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("standard deviation:", scores.std())
    
def arrayStats(arrayToInspect):
    print("min:", np.amin(arrayToInspect, axis=1))
    print("max:", np.amax(arrayToInspect, axis=1))
    
def crossValidateModel(model, X, y, name="<unknown>"):
    start = time.time()
    scores = cross_val_score(model, X, y, scoring = "neg_mean_absolute_error", n_jobs = -1, verbose = 4, cv = 10)
    end = time.time()
    elapsed_time = end - start
    print("model {0} cross_val_score took {1} seconds".format(name, elapsed_time))
    displayScores(-scores)
    
def columnsWithMissingData(X, threshold = 0.9):
    # check for null items
    null_df = X.columns[X.isnull().any()]
    null_count = X[null_df].isnull().sum()/len(X.index)
    null_count_above_threshold = null_count.loc[null_count > threshold]
    null_count_above_threshold
    
    #percentage of zero values for each numeric variable
    zero_df = X.columns[(X == 0).any()]
    zero_count = (X[zero_df] == 0).sum()/len(X.index)
    zero_count_above_threshold = zero_count.loc[zero_count > 0.9]
    return pd.concat([null_count_above_threshold, zero_count_above_threshold])


['KitchenQual', 'GarageCond', 'GarageQual', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'LotConfig', 'Utilities', 'LandSlope', 'LotShape', 'GarageType', 'BldgType', 'CentralAir', 'MSZoning', 'LandContour', 'PavedDrive']


In [3]:
# load housing data
iowa_file_path = '../data/train.csv'
home_data = pd.read_csv(iowa_file_path)

In [4]:
# drop outliers
home_data = home_data.drop(home_data['LotFrontage']
                                     [home_data['LotFrontage']>200].index)
home_data = home_data.drop(home_data['LotArea']
                                     [home_data['LotArea']>100000].index)
home_data = home_data.drop(home_data['BsmtFinSF1']
                                     [home_data['BsmtFinSF1']>4000].index)
home_data = home_data.drop(home_data['TotalBsmtSF']
                                     [home_data['TotalBsmtSF']>6000].index)
home_data = home_data.drop(home_data['1stFlrSF']
                                     [home_data['1stFlrSF']>4000].index)
home_data = home_data.drop(home_data.LowQualFinSF
                                     [home_data['LowQualFinSF']>550].index)

In [5]:
Y = home_data["SalePrice"]
X = home_data.drop(columns = ["Id", "SalePrice"])

In [6]:
# find missing data.. remove colums that have > 90% data missing
missingData = columnsWithMissingData(X, threshold = 0.9)
print("following columns have greater than 90% data missing or NULL \n",missingData)
print("original df shape = ", X.shape)
X = X.drop(columns = missingData.keys())
print("final df shape = ", X.shape)

following columns have greater than 90% data missing or NULL 
 Alley           0.937371
PoolQC          0.995871
MiscFeature     0.964212
LowQualFinSF    0.982794
BsmtHalfBath    0.944253
3SsnPorch       0.983482
ScreenPorch     0.920853
PoolArea        0.995871
MiscVal         0.965588
dtype: float64
original df shape =  (1453, 79)
final df shape =  (1453, 70)


In [7]:
# perform feature scaling for numerical features
# Steps: 1. scale columns requiring log tranformations & feature scaling
#        2. scale columns requiring normal numerical & feature scaling
#        3. concatenate the results of 1 & 2 and then apply PCA for dimensionality reduction
#        
numeric_features = X.select_dtypes(exclude=object) 
num_features_names = numeric_features.columns

# features that need a log transformation
log_features_names = ["LotFrontage", "LotArea", "1stFlrSF", "GrLivArea", "OpenPorchSF"]

log_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('logscaler', FunctionTransformer(np.log1p, validate=False)),
    ('scaler', StandardScaler())])

#numeric features that require a normal transformation
numeric_features_names = [x for x in num_features_names if x not in log_features_names]

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

num_transformers=[
        ('log', log_pipeline, log_features_names),
        ('num', numeric_pipeline, numeric_features_names),  
    ]

# ensure that result is always a dense matrix
num_col_transformer = ColumnTransformer(transformers=num_transformers, sparse_threshold = 0)

pca_pipeline = Pipeline(steps = [
                       ('num_ct', num_col_transformer), # apply the columntransformation
                       ('pca', PCA(0.99)) # apply PCA to data (log + numeric)
                       ])

X_pca_t = pca_pipeline.fit_transform(X)

In [8]:
cat_features_names = X.select_dtypes(include=object).columns
#cat_features_names = CAT_COLS

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ensure that result is always a dense matrix
X_cat_t = cat_pipeline.fit_transform(X[cat_features_names])

In [9]:
# concatenate the numerical and one hot encoded categorical data
Xt = np.concatenate((X_pca_t, X_cat_t.todense() ), axis = 1)
print("concatenated Xt shape: ", Xt.shape)

concatenated Xt shape:  (1453, 281)


In [10]:
ridge_reg = Ridge(alpha = 0.1, solver="auto")
crossValidateModel(ridge_reg, Xt, Y, "ridge")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


model ridge cross_val_score took 2.283888339996338 seconds
Scores: [16328.8001925  16819.2430448  17987.67410304 20633.67588252
 21047.51060786 19193.91308135 16735.25650182 16921.5735039
 20831.46930085 15829.70679351]
Mean: 18232.882301215395
standard deviation: 1917.912273329758


[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    2.1s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.2s finished


In [11]:
lasso_reg = Lasso(alpha = 0.1)
crossValidateModel(lasso_reg, Xt, Y, "lasso")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    2.8s remaining:    1.8s


model lasso cross_val_score took 4.335366249084473 seconds
Scores: [16446.5988932  17018.24232882 18126.63417991 20574.5655425
 21290.39285962 19659.14925501 16631.38867896 17037.54285451
 20974.8755088  15830.44380414]
Mean: 18358.983390548485
standard deviation: 1966.427645011537


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.2s finished


In [12]:
elastic_net= ElasticNet(alpha = 0.2, l1_ratio = 0.9)
crossValidateModel(elastic_net, Xt, Y, "elasticnet")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


model elasticnet cross_val_score took 0.6063768863677979 seconds
Scores: [16214.69443303 15751.86011982 16250.64452887 19033.42175816
 18077.29522209 14639.25458222 14631.87224919 14843.46069906
 19075.20971853 15640.61148048]
Mean: 16415.83247914543
standard deviation: 1631.1096218551395


[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.5s finished


In [13]:
# elastic net seems to give the best scores. use a gridsearchCV to find the best elasticNet parameters
param_grid = [
    {'alpha' : [0.2, 0.5, 0.7,  1.0], 'l1_ratio': [ 0.1,0.4,  0.5, 0.7, 0.8, 0.9]}
]

grid_search = GridSearchCV(elastic_net, param_grid, scoring = "neg_mean_absolute_error", n_jobs = -1, verbose = 10, cv = 10)
grid_search.fit(Xt, Y)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1646s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 172 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    8.3s finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=0.2, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.9, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'alpha': [0.2, 0.5, 0.7, 1.0],
                          'l1_ratio': [0.1, 0.4, 0.5, 0.7, 0.8, 0.9]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=10)

In [14]:
grid_search.best_params_


{'alpha': 0.2, 'l1_ratio': 0.9}

In [17]:
# use the grid serach results to create the final predictor for the test
elastic_net= ElasticNet(alpha = 0.2, l1_ratio = 0.9)
elastic_net.fit(Xt,Y)

#path to file you will use for predictions
test_data_path = '../data/test.csv'
test_data = pd.read_csv(test_data_path)

X_test = test_data.drop(columns = ["Id"])
X_test = test_data.drop(columns = missingData.keys())

X_pca_test = pca_pipeline.transform(X_test)
# categorical colums that are OHE
X_cat_test = cat_pipeline.transform(X_test[categorical_features_names])

X_final_test = np.concatenate((X_pca_test, X_cat_test.todense() ), axis = 1)
#make predictions which we will submit. 
y_pred = elastic_net.predict(X_final_test)

#The lines below shows how to save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': y_pred})
output.to_csv('../data/submission.csv', index=False)

ValueError: Column ordering must be equal for fit and for transform when using the remainder keyword

In [ ]:
# use the grid serach results to create the final predictor for the test
elastic_net= clf = RandomForestRegressor(random_state=1, n_estimators = 500, criterion="mae", n_jobs=-1)
elastic_net.fit(Xt,Y)

#path to file you will use for predictions
test_data_path = '../data/test.csv'
test_data = pd.read_csv(test_data_path)

X_test = test_data.drop(columns = ["Id"])
X_test = test_data.drop(columns = missingData.keys())

# numerical columns
X_num_test = ct.fit_transform(X_test)
# PCA on the numerical data
X_pca_test = pca.transform(X_num_test)
# categorical colums that are OHE
X_cat_test = cat_pipeline.transform(X_test[categorical_features_names])

X_final_test = np.concatenate((X_pca_test, X_cat_test.todense() ), axis = 1)
#make predictions which we will submit. 
y_pred = elastic_net.predict(X_final_test)

#The lines below shows how to save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': y_pred})
output.to_csv('../data/submission.csv', index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

est=GradientBoostingRegressor(n_estimators=400, max_depth=5, loss='ls',min_samples_split=2,learning_rate=0.1, verbose = 5).fit(Xt, np.log(Y))
y_pred = est.predict(X_final_test)

#The lines below shows how to save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': np.exp(y_pred)})
output.to_csv('../data/submission.csv', index=False)

In [ ]:
# use random forest regressor 
random_forest = RandomForestRegressor(random_state=1, n_estimators = 500, criterion="mae", n_jobs=-1)
crossValidateModel(random_forest, Xt, Y, "random_forest")